<a href="https://colab.research.google.com/github/ethvedbitdesjan/SummerResearch/blob/main/SwinBase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%%capture
!pip install datasets transformers
!pip install wandb -qU
!pip install weightwatcher
!pip install accelerate -U

In [2]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
%env WANDB_PROJECT=funny_not_funny_Swin

env: WANDB_PROJECT=funny_not_funny_Swin


In [5]:
from google.colab import drive
drive.mount('content/')

Mounted at content/


In [ ]:
%cd content/MyDrive/SummerResearch

In [7]:
from datasets import load_dataset

dataset = load_dataset("imagefolder", data_dir="Dataset/")

Resolving data files:   0%|          | 0/1615 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/539 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/539 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

...

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset imagefolder downloaded and prepared to /root/.cache/huggingface/datasets/imagefolder/default-78db231af1852f96/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
# Critical imports
import os
import numpy as np
import pandas as pd
from PIL import Image
import random
import cv2
import copy
import torch
import torchvision
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.optim as optim
from torch.optim import lr_scheduler
import weightwatcher as ww

In [16]:
from transformers import AutoImageProcessor
model_name_or_path = 'microsoft/swinv2-base-patch4-window16-256'
image_processor  = AutoImageProcessor.from_pretrained(model_name_or_path)
image_processor

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 256,
    "width": 256
  }
}

In [17]:
def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = image_processor([x for x in example_batch['image']], return_tensors='pt')

    # Don't forget to include the labels!
    inputs['labels'] = example_batch['label']
    return inputs

prepared_ds = dataset.with_transform(transform)

In [18]:
import numpy as np
from datasets import load_metric

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

<ipython-input-18-223a1e4a98a9>:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [20]:
from transformers import AutoImageProcessor, Swinv2ForImageClassification

labels = prepared_ds['train'].features['label'].names
model = Swinv2ForImageClassification.from_pretrained(model_name_or_path, num_labels=len(labels), id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}, ignore_mismatched_sizes=True)

NUM_CLASSES = 2
BATCH_SIZE = 8
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Some weights of Swinv2ForImageClassification were not initialized from the model checkpoint at microsoft/swinv2-base-patch4-window16-256 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 1024]) in the checkpoint and torch.Size([2, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./swin-base",
  per_device_train_batch_size=BATCH_SIZE,
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=True,
  save_steps=200,
  eval_steps=200,
  logging_steps=10,
  learning_rate=1e-5,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='wandb',
  load_best_model_at_end=True,
  optim='adamw_torch',
  weight_decay=0.1
)

In [22]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["validation"],
    tokenizer=image_processor,
)

In [23]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

wandb: Currently logged in as: vedaantj (uiuc-disruption-lab). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Accuracy
200,0.614700,0.654909,0.588126
400,0.541700,0.640984,0.634508
600,0.428400,0.642268,0.653061
800,0.406900,0.639068,0.658627


***** train metrics *****
  epoch                    =         4.0
  total_flos               = 616714556GF
  train_loss               =      0.5394
  train_runtime            =  0:12:46.29
  train_samples_per_second =        8.43
  train_steps_per_second   =       1.054


In [24]:
watcher = ww.WeightWatcher(model=model)
details = watcher.analyze()
summary = watcher.get_summary(details)
summary

{'log_norm': 2.879,
 'alpha': 3.5156382839849285,
 'alpha_weighted': 5.529467850719167,
 'log_alpha_norm': 5.791193466861529,
 'log_spectral_norm': 1.659,
 'stable_rank': 22.73}

In [25]:
metrics = trainer.evaluate(prepared_ds['validation'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =        4.0
  eval_accuracy           =     0.6586
  eval_loss               =     0.6391
  eval_runtime            = 0:00:29.55
  eval_samples_per_second =     18.239
  eval_steps_per_second   =      2.301


In [26]:
metrics = trainer.predict(prepared_ds['test'])
trainer.log_metrics("eval", metrics.metrics)
trainer.save_metrics("eval", metrics.metrics)
wandb.finish()

***** eval metrics *****
  test_accuracy           =     0.6456
  test_loss               =     0.6594
  test_runtime            = 0:00:34.32
  test_samples_per_second =     15.702
  test_steps_per_second   =      1.981


eval/accuracy,▁▆▇██
eval/loss,█▂▂▁▁
eval/runtime,█▃▁▁▂
eval/samples_per_second,▁▆██▆
eval/steps_per_second,▁▆██▆
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,▇▇█▇▇▇▆▇▆▆▆▅▄▅▇▅▃▆▃▄▃▃▄▃▄▃▂▄▃▂▄▃▂▁▃▄▁▃▂▂
train/total_flos,▁
train/train_loss,▁


In [27]:
predictions_tensor = torch.tensor(metrics.predictions)
preds = torch.sigmoid(predictions_tensor)
print(preds.shape)
preds = torch.argmax(preds, dim=1)
preds, preds.shape

torch.Size([539, 2])


(tensor([1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1,
         1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
         1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
         0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0,
         0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
         0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1,
         0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0,
         0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1,
         0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
         1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
         1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 0, 1, 1, 1, 1, 1

In [31]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

In [32]:
print(accuracy_score(preds, metrics.label_ids), f1_score(preds, metrics.label_ids))
pd.DataFrame(confusion_matrix(preds, metrics.label_ids), columns=['Test Funny', 'Test Not Funny'], index=['Pred Funny', 'Pred Not Funny'])

0.6456400742115028 0.7066052227342551


,Test Funny,Test Not Funny
Pred Funny,118,84
Pred Not Funny,107,230
